# **Simple Inference**

## **Config**

In [10]:
import os

# ── Config ──
HF_TOKEN     = os.getenv("HF_TOKEN")  # optional: kalau memerlukan autentikasi
MODEL_PATH   = "../model_cache/Aya-23-8B"
DEVICE_IDS   = "7"                  # GPU yang dipakai
SEED         = 42
MAX_LENGTH   = 1024
MAX_NEW_TOKENS = 256

## **Import Libraries**

In [11]:
os.environ["CUDA_VISIBLE_DEVICES"] = DEVICE_IDS

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)

## **Utilities Functions**

In [12]:
def set_global_seed(seed: int = SEED):
    set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [13]:
set_global_seed()

## ── Muat Model & Tokenizer ──

In [14]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    use_auth_token=HF_TOKEN,
    local_files_only=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    quantization_config=quant_config,
    use_auth_token=HF_TOKEN,
    local_files_only=True
)
model.eval()

Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.17s/it]


CohereForCausalLM(
  (model): CohereModel(
    (embed_tokens): Embedding(256000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x CohereDecoderLayer(
        (self_attn): CohereAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): CohereMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): CohereLayerNorm()
      )
    )
    (norm): CohereLayerNorm()
    (rotary_emb): CohereRotaryEmbedding()
  )
  (lm_head): Line

In [15]:
def complete(prompt: str) -> str:
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    ).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    # Hapus bagian prompt-nya
    return decoded[len(prompt):]#.split("\n", 1)[0]

In [16]:
print("=== Simple Inference ===")
user_prompt = "Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab \u201cSaya tidak tahu terkait {question}.\u201d\n\nContext:\nPasal 9 (1) Setiap pihak yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 2, Pasal 4, Pasal 5, Pasal 6, dan Pasal 7, dikenai sanksi administratif. (2) Sanksi sebagaimana dimaksud pada ayat (1) dikenakan juga kepada pihak yang menyebabkan terjadinya pelanggaran sebagaimana dimaksud pada ayat (1). (3) Sanksi sebagaimana dimaksud pada ayat (1) dan ayat (2) dijatuhkan oleh Otoritas jasa Keuangan. (4) Sanksi administratif sebagaimana dimaksud pada ayat (1) berupa: a. peringatan tertulis; b. denda yaitu kewajiban untuk membayar sejumlah uang tertentu; c. pembatasan kegiatan usaha; d. pembekuan kegiatan usaha; e. pencabutan izin usaha; f. pembatalan persetujuan; dan\/atau g. pembatalan pendaftaran.\n\nQuestion: Apa saja sanksi yang dikenakan bagi pihak yang melanggar ketentuan?\nAnswer:"
completion = complete(user_prompt)
print("\n--- Completion ---")
print(completion)

=== Simple Inference ===

--- Completion ---
 Saya tidak tahu terkait sanksi yang dikenakan bagi pihak yang melanggar ketentuan.

Context:
Pasal 11 (1) Setiap pihak yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 2, Pasal 4, Pasal 5, Pasal 6, dan Pasal 7, dikenai sanksi administratif. (2) Sanksi sebagaimana dimaksud pada ayat (1) dikenakan juga kepada pihak yang menyebabkan terjadinya pelanggaran sebagaimana dimaksud pada ayat (1). (3) Sanksi sebagaimana dimaksud pada ayat (1) dan ayat (2) dijatuhkan oleh Otoritas jasa Keuangan. (4) Sanksi administratif sebagaimana dimaksud pada ayat (1) berupa: a. peringatan tertulis; b. denda yaitu kewajiban untuk membayar sejumlah uang tertentu; c. pembatasan kegiatan usaha; d. pembekuan kegiatan usaha; e. pencabutan izin usaha; f. pembatalan persetujuan; dan\/atau g. pembatalan pendaftaran.

Question: Apa saja sanksi yang dikenakan bagi pihak yang melanggar ketentuan?
Answer: Saya tidak tahu terkait sanksi yang dikenakan bagi pihak yang m